In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsbombpy import sb
from mplsoccer import Pitch,VerticalPitch
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

In [2]:
euro24=list(sb.matches(season_id=282,competition_id=55)['match_id'])
euro24_matches=sb.matches(season_id=282,competition_id=55)

In [3]:
euro24_matches=euro24_matches[['match_id','home_team','home_score','away_team','away_score','competition_stage']]

In [4]:
df_main=pd.DataFrame()
for i in euro24:
    data=sb.events(match_id=i)
    df_main=pd.concat([data,df_main], ignore_index=True)

In [5]:
df_shots=df_main[df_main.type=='Shot']

In [6]:
df_shots=df_shots[['id','timestamp','match_id','player','team','shot_freeze_frame','shot_type','shot_outcome','shot_statsbomb_xg','location','shot_end_location']]



In [7]:
df_shots.match_id.unique()

array([3930158, 3930159, 3930160, 3930162, 3930163, 3930166, 3930169,
       3930170, 3930174, 3930175, 3930176, 3930178, 3930181, 3930183,
       3938638, 3938643, 3938644, 3938641, 3938639, 3938642, 3938640,
       3938637, 3930161, 3930164, 3930165, 3930168, 3930167, 3930172,
       3930173, 3930177, 3940878, 3940983, 3930179, 3930182, 3941017,
       3941018, 3941019, 3941020, 3941021, 3941022, 3930184, 3938645,
       3942226, 3942227, 3930171, 3930180, 3942349, 3942382, 3942752,
       3943043, 3942819])

In [8]:
euro24_matches['match']=euro24_matches['competition_stage']+' : '+euro24_matches['home_team']+' vs '+euro24_matches['away_team']
euro24_matches

,match_id,home_team,home_score,away_team,away_score,competition_stage,match
0,3942819,Netherlands,1,England,2,Semi-finals,Semi-finals : Netherlands vs England
1,3943043,Spain,2,England,1,Final,Final : Spain vs England
2,3942752,Spain,2,France,1,Semi-finals,Semi-finals : Spain vs France
3,3942382,Netherlands,2,Turkey,1,Quarter-finals,Quarter-finals : Netherlands vs Turkey
4,3942349,Portugal,0,France,0,Quarter-finals,Quarter-finals : Portugal vs France
5,3930180,Netherlands,2,Austria,3,Group Stage,Group Stage : Netherlands vs Austria
6,3930171,Denmark,1,England,1,Group Stage,Group Stage : Denmark vs England
7,3942227,England,1,Switzerland,1,Quarter-finals,Quarter-finals : England vs Switzerland
8,3942226,Spain,2,Germany,1,Quarter-finals,Quarter-finals : Spain vs Germany
9,3938645,Ukraine,0,Belgium,0,Group Stage,Group Stage : Ukraine vs Belgium


In [9]:
df_shots=pd.merge(df_shots,euro24_matches[['match_id','match']],on='match_id',how='left')

In [15]:



# Get unique matches
unique_matches = euro24_matches['match'].unique().tolist()

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=unique_matches,
    description='Select Match:',
)

# Initialize input_match_id
input_match_id = None

# Function to handle the selection
def on_select(change):
    global input_match_id  # Use global to modify the variable outside the function
    input_match_id = change['new']
    print(f'input_match_id: {input_match_id}')

# Attach the event handler
dropdown.observe(on_select, names='value')

# Display the dropdown
display(dropdown)


Dropdown(description='Select Match:', options=('Semi-finals : Netherlands vs England', 'Final : Spain vs Engla…

In [27]:
input_match_id

'Round of 16 : Portugal vs Slovenia'

In [29]:
all_shots=pd.DataFrame()
selected_match_events=df_shots[df_shots.match==(input_match_id)].reset_index(drop=True)
selected_matches=euro24_matches[euro24_matches.match==(input_match_id)].reset_index(drop=True)

for i in selected_match_events.index:
    
    data = selected_match_events.at[i, 'shot_freeze_frame']
    
    # Check if data is a list, otherwise skip the iteration
    if isinstance(data, list):
        # Convert to DataFrame
        df = pd.DataFrame([{
            'location_x': item['location'][0],
            'location_y': item['location'][1],
            'player_name': item['player']['name'],
            'teammate': item['teammate']
        } for item in data])
        
        event_info_df = pd.DataFrame([selected_match_events.iloc[i]])

        # Repeat the DataFrame to match the length of 'df'
        event_info_df = pd.concat([event_info_df] * df.shape[0], ignore_index=True)
        
        
        # Concatenate row data with the event information in `selected_match_events.iloc[i]`
        df = pd.concat([event_info_df, df], axis=1)
        
        # Append the concatenated result to `all_goals`
        all_shots = pd.concat([all_shots, df], axis=0, ignore_index=True)

# Display the final concatenated DataFrame
# Function to convert lists to tuples, leave other types unchanged
def convert_to_tuple(x):
    if isinstance(x, list):
        return tuple(x)
    return x

all_shots['shot_end_location'] = all_shots['shot_end_location'].apply(convert_to_tuple)
all_shots['location'] = all_shots['location'].apply(convert_to_tuple)

# Extract x, y, endx, endy coordinates
all_shots['X'] = all_shots['location'].apply(lambda loc: loc[0])
all_shots['Y'] = all_shots['location'].apply(lambda loc: loc[1])
all_shots['endX'] = all_shots['shot_end_location'].apply(lambda loc: loc[0])
all_shots['endY'] = all_shots['shot_end_location'].apply(lambda loc: loc[1])

# Use None as a fallback if 'z' is missing
all_shots['endZ'] = all_shots['shot_end_location'].apply(lambda loc: loc[2] if len(loc) > 2 else None)
all_shots.drop(columns={'shot_freeze_frame','location','shot_end_location'},inplace=True)


In [39]:
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import plotly.express as px
from IPython.display import display
import ipywidgets as widgets

def plot_shots(shot_id):
    # Filter shots for the selected ID
    df_each_shoot = all_shots[all_shots.id == shot_id].reset_index(drop=True)
    
    # Extract unique values for display
    df_each_shoot_time = df_each_shoot.timestamp.unique()[0]
    df_each_shoot_shooter = df_each_shoot.player.unique()[0]
    df_each_shoot_shot_outcome = df_each_shoot.shot_outcome.unique()[0]
    df_each_shoot_team = df_each_shoot.team.unique()[0]
    # Create the pitch
    pitch = Pitch(pitch_type='statsbomb', pitch_color='black', line_color='white')
    fig, ax = pitch.draw(figsize=(16, 11), constrained_layout=True, tight_layout=False)
    fig.set_facecolor("black")
    ax.set_xlim(75, 130)

    for i in range(len(df_each_shoot)):
        # Plot the shot lines
        plt.plot((df_each_shoot.iloc[i]['X'], df_each_shoot.iloc[i]['endX']),
                 (df_each_shoot.iloc[i]['Y'], df_each_shoot.iloc[i]['endY']),
                 color="yellow", ls='--')
        
        # Scatter plot for the shots
        plt.scatter(df_each_shoot.iloc[i]['X'], df_each_shoot.iloc[i]['Y'], color='yellow', marker='o')
        
        # Location markers based on teammate status
        if df_each_shoot.iloc[i]['teammate']:
            plt.scatter(df_each_shoot.iloc[i]['location_x'], df_each_shoot.iloc[i]['location_y'], color='green')
        else:
            plt.scatter(df_each_shoot.iloc[i]['location_x'], df_each_shoot.iloc[i]['location_y'], color='red')
    
    # Add dummy points for legend only
    plt.scatter([], [], color='yellow', marker='o', label='Shooter')
    plt.scatter([], [], color='green', marker='o', label='Attacking')
    plt.scatter([], [], color='red', marker='o', label='Defending')

    # Create a separate plot for endY vs. endZ if needed
    if df_each_shoot.endZ.notnull().any():  # Ensure there are valid endZ values
        fig_scatter = px.scatter(
            df_each_shoot,
            x='endY',
            y='endZ',
            color='shot_outcome',
            title='Shot Outcome Location',
            labels={'shot_outcome':'Shot Outcome','endY': 'End Y', 'endZ': 'End Z','endX':'End X','player':'Player','team':'Team','player':'Player','match_id':'Match id',
                  'timestamp':'Timestamp', 'shot_statsbomb_xg':'StatsBomb xG'},
            hover_data={'endX': True,
                        'player': True,  
                        'team': True,         
                        'match_id': True,     
                        'timestamp': True,
                        'shot_statsbomb_xg': True}
        )
        
        # Adding the vertical and horizontal lines
        fig_scatter.add_shape(type="line", x0=36, x1=36, y0=0, y1=4, line=dict(color="red", width=2, dash="dash"))
        fig_scatter.add_shape(type="line", x0=44, x1=44, y0=0, y1=4, line=dict(color="red", width=2, dash="dash"))
        fig_scatter.add_shape(type="line", x0=30, x1=50, y0=2.66, y1=2.66, line=dict(color="red", width=2, dash="dash"))
        fig_scatter.add_shape(type="line", x0=30, x1=50, y0=0, y1=0, line=dict(color="red", width=2, dash="dash"))
        
        # Update layout to make it equal aspect ratio and set limits
        fig_scatter.update_layout(
            xaxis=dict(scaleanchor="y", constrain="domain", range=[30, 50]),
            yaxis=dict(constrain="domain", range=[0, 4])
        )

        fig_scatter.show()

    # Title with formatted match information
    plt.title(f"{selected_matches['competition_stage'].iloc[0]}: "
              f"{selected_matches['home_team'].iloc[0]} "
              f"({selected_matches['home_score'].iloc[0]}) - "
              f"{selected_matches['away_team'].iloc[0]} "
              f"({selected_matches['away_score'].iloc[0]})", color='white')

    # Add shot details as a table
    shot_details_data = [
        ["Time", df_each_shoot_time],
        ["Team", df_each_shoot_team],
        ["Player", df_each_shoot_shooter],
        ["Shot Outcome", df_each_shoot_shot_outcome]
    ]
    column_labels = ["Detail", "Value"]

    # Create the shot details table
    details_table = ax.table(cellText=shot_details_data, colLabels=column_labels,
                             cellLoc='center', loc='left', colColours=["#1f77b4", "#ff7f0e"])
    details_table.auto_set_font_size(False)
    details_table.set_fontsize(12)
    details_table.scale(1, 1.4)

    # Customize the table appearance (optional)
    for key, cell in details_table.get_celld().items():
        cell.set_edgecolor('black')
        cell.set_text_props(color='Black')

    # Legend for player positions
    ax.legend(loc='lower right')

    # Calculate and display team shot statistics as a table
    team_shots = selected_match_events.groupby('team')['team'].count().reset_index(name='shots')
    table_data = team_shots.values
    column_labels = ["Team", "Total Shots"]

    # Create and display the team shots table
    table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='right', colColours=["#1f77b4", "#ff7f0e"])
    table.auto_set_font_size(False)
    table.set_fontsize(13)
    table.scale(0.4, 1.4)

    # Show the plot
    plt.show()

# Create a dropdown widget
shot_id_dropdown = widgets.Dropdown(
    options=all_shots['id'].unique(),
    description='Shot ID:',
    value=all_shots['id'].unique()[0]  # Default value
)

# Display the dropdown and plot based on the selected shot ID
widgets.interactive(plot_shots, shot_id=shot_id_dropdown)


interactive(children=(Dropdown(description='Shot ID:', options=('e93c5447-8a6b-4d88-94c4-47d0ae8883ac', '6243d…